## Experimental interactive sketch

#### Imports

In [47]:
from __future__ import print_function # For py 2.7 compat
import ipywidgets as widgets
from IPython.display import Javascript
from traitlets import Unicode
from traitlets import CBytes
from traitlets import CInt
from traitlets import CBool
import numpy as np

#### Globals

In [48]:
p55_globals = {
    "canvas_width": 480,
    "canvas_height": 120
}

In [49]:
def update_javascript_globals():
    ##### Copy globals to javascript
    javascript_code = "var p55_globals = {\n"

    num_items = len(p55_globals)
    for i, k in enumerate(p55_globals):
        javascript_code += "  \"{}\": {}".format(k, p55_globals[k])
        # the dreded comma
        if i < num_items - 1:
            javascript_code += ","
        javascript_code += "\n"
    javascript_code += "};\n"
    javascript_code += "window.p55_globals = p55_globals\n"

    print(javascript_code)
    Javascript(javascript_code)

In [50]:
##### Copy globals to javascript
update_javascript_globals()

var p55_globals = {
  "canvas_width": 480,
  "canvas_height": 120
};
window.p55_globals = p55_globals



#### Python side canvas widget

In [51]:
class P55CanvasWidget(widgets.DOMWidget):
    _view_name = Unicode('P55CanvasView').tag(sync=True)
    _view_module = Unicode('p55canvas').tag(sync=True)
    value = CInt().tag(sync=True)
    a = np.zeros([p55_globals["canvas_width"],p55_globals["canvas_height"],4], dtype='uint8') + 128
    sidecar = CBytes(a.tobytes()).tag(sync=True)
    mouseX = CInt().tag(sync=True)
    mouseY = CInt().tag(sync=True)
    mousePressed = CBool().tag(sync=True)    

#### Javascript side

In [52]:
%%javascript
requirejs.undef('p55canvas');

var p55_globals = window.p55_globals;

define('p55canvas', ["jupyter-js-widgets"], function(widgets) {
    // we track all mouse movement and store into globals
    var p55_mouse_x = 0;
    var p55_mouse_y = 0;
    var p55_mouse_is_down = false;

    var callback_on_mouse_down = function(evt) {
        p55_mouse_is_down = true;
    };
    
    var callback_on_mouse_up = function(evt) {
        p55_mouse_is_down = false;
    };

    function track_mouse_pos(evt) {
        p55_mouse_x = evt.clientX;
        p55_mouse_y = evt.clientY;
    }

    var P55CanvasView = widgets.DOMWidgetView.extend({
        render: function() { 
            var that = this;
            var canvas_str = '<canvas width="' + p55_globals["canvas_width"] + '" height="' + p55_globals["canvas_height"] + '"></canvas>'
            this.$canvas = $(canvas_str);
            this.$el.append(this.$canvas);
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
            this.$canvas[0].addEventListener('mousemove', track_mouse_pos, false);
            this.$canvas[0].addEventListener('mousedown', callback_on_mouse_down, false);
            this.$canvas[0].addEventListener('mouseup', callback_on_mouse_up, false);
        },
        
        value_changed: function() {
            // anytime the value is changed we do a full sync
            var ctx = this.$canvas[0].getContext("2d");
            var buffer = this.model.get('sidecar').buffer
            var arr = new Uint8ClampedArray(buffer);
            var imdata = new ImageData(arr, p55_globals["canvas_width"], p55_globals["canvas_height"]);
            ctx.putImageData(imdata, 0, 0)
            var rect = this.$canvas[0].getBoundingClientRect();
            var local_x = Math.floor( p55_mouse_x - rect.left );
            var local_y = Math.floor( p55_mouse_y - rect.top );
            this.handle_mouse(local_x, local_y, p55_mouse_is_down);
        },
        
        handle_mouse: function(x, y, is_down) {
            this.model.set('mouseX', x);
            this.model.set('mouseY', y);
            this.model.set('mousePressed', is_down);
            this.touch();
        },
        
    });
    
    return {
        P55CanvasView: P55CanvasView
    };
});

<IPython.core.display.Javascript object>

### Try it out

In [53]:
## OK, combine with PIL
from PIL import Image, ImageDraw
import numpy as np
import IPython.display 

In [54]:
widget = P55CanvasWidget()
IPython.display.display(widget)

In [60]:
def drawLine(widget, endpoints, fill):
    widget.drawContext.line(endpoints, fill)

def wrapped_draw(draw_func):
    widget.value = 0
    for n in range(1000):
        # this is a hack to make the widget interactive while loop is processing
        for x in range(100):
            get_ipython().kernel.do_one_iteration()
        color = (0,0,0,255)
        img = Image.new('RGBA', [p55_globals["canvas_width"], p55_globals["canvas_height"]], color=color)
        widget.drawContext = ImageDraw.Draw(img)
        draw_func(widget)
        widget.sidecar = img.tobytes()
        widget.value = widget.value + 1        

In [64]:
def draw(self):
    if not self.mousePressed:
        color = (255,0,0,255)
    else:
        color = (0,0,255,255)
    drawLine(self, (0, 0) + (self.mouseX, self.mouseY), fill=color)

In [65]:
wrapped_draw(draw)